# Compteur

In [ ]:
SELECT count(*) count, TYPE
from `olivier_it`
where status_2 is null
GROUP BY TYPE

In [ ]:
-- IT_count
select COUNT(STATUS_2), status_2
from `olivier_it`
GROUP BY status_2
ORDER BY COUNT(status_2) desc

# VIEW

### `OK-POSITION : NB OF MATCH`

Version condensée de IT_Equipment + jointure avec asset_new uniquement sur la position

In [ ]:
CREATE VIEW IT_Equipment_match_position_reduced_v3
as
SELECT it.olivier_index, count(it.olivier_index) as RowNumber

FROM   `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Salle_origin = aa.Salle
AND
	it.rack = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
GROUP BY it.olivier_index

In [ ]:
CREATE VIEW IT_Equipment_match_position_v3
AS
SELECT it.Long_name,
		it.short_name,
        it.asset_id,
        it.Salle_Origin,
        it.rack,
        it.slot_u,
        it.olivier_index,
        it.PROJET_Responsable,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.etat, it2.RowNumber

       
FROM   `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Salle_Origin = aa.Salle
AND
	it.rack = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v2` it2
ON it.olivier_index = it2.olivier_index

WHERE it2.rowNumber > 1
ORDER BY it2.rowNumber desc, olivier_index, it.status_2 ASC

## `long_name DU FABRICANT : ASSET_ID`

### netapp & ATTO & Brocade

System 2-en-1 avec chassis + server. Y aura 2 objects differents alors que c'est la meme chose, on va donc s'assurer que le match se fait sur le chassis plutot que sur le server + on rattache tous les serveurs à au bon chassis

In [ ]:
CREATE VIEW it_built_system_NAS_v2
AS
SELECT rack,
       Salle_Origin,
       status_2,
       asset_id,
       long_name, 
        model,
        manufacturer,
        olivier_index
FROM   `olivier_it` it
WHERE  status_2 IS NOT NULL and (it.manufacturer like 'netapp%' or it.manufacturer like 'ATTO%' or it.manufacturer like 'Brocade%' ) and ASSET_ID != '0'
ORDER  BY rack DESC

### EXADATA

In [ ]:
CREATE VIEW it_build_system_EXADATA_v2
as
SELECT rack,
       Salle_Origin,
       status_2,
       asset_id,
       long_name, 
        model,
        manufacturer,
        olivier_index
FROM   `olivier_it` it
where it.manufacturer like '%oracle%' and asset_id != '0'
ORDER  BY rack DESC

### HPC

In [ ]:
CREATE VIEW it_build_system_HPC_v2
as
SELECT rack,
       Salle_Origin,
       status_2,
       asset_id,
       long_name, 
        model,
        manufacturer,
        olivier_index,
        (LEFT(long_name,6)) ROOT_FERME
FROM   `olivier_it` it
where it.long_name like '%HPC%' or it.long_name like '%HPC%'
ORDER BY rack DESC, Salle_Origin

### HANNA

In [ ]:
CREATE VIEW it_build_system_HANNA_v2
as
SELECT rack,
       Salle_Origin,
       status_2,
       asset_id,
       long_name, 
        model,
        manufacturer,
        olivier_index
        
FROM   `olivier_it` it
where (it.rack like '%hana%' OR it.long_name like '%hana%')  and asset_id != '0'
ORDER BY rack DESC, Salle_Origin

### RING

In [ ]:
CREATE VIEW it_build_system_RING_v2
as
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name, 
        it.model,
        it.manufacturer,
        it.olivier_index
       
       
FROM   `olivier_it` it
WHERE (it.short_name like '%NA9TCR%' 
OR it.long_name like '%NA9TCR%') and (it.status_2 is not null or it.asset_id != '0')	
GROUP BY rack, Salle_Origin
ORDER BY it.rack DESC, it.Salle_Origin

### FEX

In [ ]:
CREATE VIEW IT_Equipment_FEX_v2
as
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name, 
        it.model,
        it.manufacturer,
        it.olivier_index,
        (LEFT(it.long_name,8)) ROOT_FEX
from `olivier_it` it
where Fabricant	 like '%cisco%' and ( it.long_name = (LEFT(it.long_name,8)) )
ORDER BY olivier_index asc

## `'CMDB-OK-OLD'`

In [ ]:
CREATE VIEW IT_Equipment_OLD_v2
AS
SELECT it.rack,
		aa.nom_court,
        it.short_name,
        aa.nom_long,
        it.long_name,
        long_name,
       it.Salle_Origin,
       it.status_2,
       REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','') new_nom_court,
		REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','') new_nom_long
        

from `olivier_it` it
LEFT JOIN `assets_filtree_v2` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.long_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.long_name 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name

where aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD'

## `HITASHI`

In [ ]:
CREATE VIEW IT_Equipment_cleaning_Hitachi_v2
as
SELECT it.rack,
        it.short_name it_short_name,
        it.long_name it_nom,
       it.Salle_Origin,
       it.status_2,
       (CASE WHEN LOCATE(':',it.long_name) > 0 then 
       UPPER(REPLACE(REPLACE(LEFT(it.long_name, LOCATE(':',it.long_name)-1),'_','-'),'-',''))
       else it.long_name
       END) clean_name

from `olivier_it` it 
WHERE Fabricant like '%Hitachi%'
ORDER BY it.rack DESC, it.Salle_Origin

----------------------

# SELECT

----------------------

### QUERY DE BASE

In [ ]:
SELECT it.asset_id asset_id_it, it.short_name, it.long_name, it.status_2 status_it, it.model, it.olivier_index, it.hauteur_u 
FROM `olivier_it` it

In [ ]:
SELECT Type, Etat_dinstallation,it.asset_id asset_id_it, it.short_name, it.long_name, it.status_2 status_it, it.model, it.olivier_index, it.hauteur_u 
FROM `olivier_it` it
where status_2 is null
ORDER BY olivier_index asc
limit 500

In [ ]:
SELECT oit.Long_name,
		oit.short_name,
        oit.asset_id,
        oit.Salle_Origin,
        oit.rack,
        oit.slot_u
        oit.olivier_index,
        oit.PROJET_Responsable

FROM `olivier_it` oit

### `OK-CMDB-ASSET-ID`

In [ ]:
SELECT it.Long_name,
		it.short_name,
        it.asset_id,
        aa.nom_court,
        it.Salle_Origin,
        it.Rack_U_Origin,
        it.olivier_index,
        it.PROJET_Responsable

FROM `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa on aa.asset_id = it.asset_id and it.asset_id != '0'

### `'ASSET_ID OK / SHORT_NAME MISMATCH'`

In [ ]:
SELECT it.Long_name,
		it.short_name,
        it.asset_id,
        aa.nom_court,
        it.Salle_Origin,
        it.Rack_U_Origin,
        it.olivier_index,
        it.PROJET_Responsable

FROM `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa 
ON aa.nom_long = it.long_name or (aa.nom_court = it.short_name and it.short_name !=0)

### `Asset_ID affiché`

Seulement pour les serveurs, match entre les noms courts de IT_equipment et celui 
d'assets avec les 3 derniers caracteres en moins. Exemple : IT_equipment.long_name = SU941, assets.nom_court = SU941AOS 

##### 0 resultat affiché, ils sont tous contenu dans le OK-CMDB-ASSET-ID

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.long_name, aa.nom_court, aa.nom_long, it.model, it.reperage_id, it.status_2 FROM `olivier_it` it
INNER JOIN `assets_filtree_v2` aa on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name

where it.type = 'Serveurs' and it.status_2 is null

### `OK-POSITION : NB OF MATCH`

HORS model qui contiennent : BL460C BL660C BL20P BL860C BL420C BL680C WS460C BL870C

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.long_name, aa.nom_court, aa.nom_long, it.model, aa.model, it.olivier_index, aa.dalle, it2.rownumber
FROM `olivier_it` it 
INNER JOIN `assets_filtree_v2`aa
ON (
	it.Salle_Origin = aa.Salle
AND
	it.rack = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v2` it2
ON it.olivier_index = it2.olivier_index
WHERE it.status_2 is null
AND aa.model not like '%BL460C%'
AND aa.model not like '%BL660C%'
AND aa.model not like '%BL20P%'
AND aa.model not like '%BL860C%'
AND aa.model not like '%BL420C%'
AND aa.model not like '%BL680C%'
AND aa.model not like '%WS460C%'
AND aa.model not like '%BL870C%'
ORDER BY rownumber desc

## `long_name DU FABRICANT : ASSET_ID`

### NetApp

In [ ]:
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, status_2
from `assets_filtree_v2` aa
where manufacturer like 'NETAPP%'
order by dalle asc

---
select it.asset_id asset_id_id, it.short_name, it.long_name, it.status_2 status_it, it.manufacturer, it.model, it.olivier_index, Salle_Origin, rack, Slot_U, Hauteur_U
from `olivier_it` it
where manufacturer like 'NETAPP%'

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it2.status_2 status_2,
       it.asset_id,
       it.long_name, 
       it.model,
       it.manufacturer,
       it.olivier_index,
       ( CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.manufacturer, ' : ', it2.asset_id) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
       
FROM   `olivier_it` it
 LEFT JOIN `it_built_system_NAS_v2` it2
 ON        it2.rack = it.rack 
 AND       it2.Salle_Origin = it.Salle_Origin
 AND		  it2.manufacturer = it.manufacturer

WHERE it.status_2 is null and (it.manufacturer like 'netapp%' or it.manufacturer like 'ATTO%' or it.manufacturer like 'Brocade%' )
ORDER  BY it.rack DESC

### Exadata

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it2.status_2 status_2,
       it.asset_id,
       it.long_name, 
       it.model,
       it.manufacturer,
       it.olivier_index,
       ( CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.manufacturer, ' : ', it2.asset_id) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
       
FROM   `olivier_it` it
 LEFT JOIN `it_build_system_EXADATA_v2` it2
 ON        it2.rack = it.rack 
 AND       it2.Salle_Origin = it.Salle_Origin
 AND		  it2.manufacturer = it.manufacturer

WHERE it.status_2 is null and (it.manufacturer like '%oracle%')
ORDER  BY it.rack DESC

### HPC

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it2.status_2 status_2,
       it.asset_id,
       it.long_name, 
       it.model,
       it.manufacturer,
       it.olivier_index,
       ( CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.manufacturer, ' : ', it2.root_ferme) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
       
FROM   `olivier_it` it
 LEFT JOIN `it_build_system_HPC_v2` it2
 ON (LEFT(it.long_name,6)) = it2.root_ferme

WHERE it.status_2 is null and (it.manufacturer like '%oracle%')
ORDER  BY it.rack DESC

### HANNA

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it2.status_2 status_2,
       it.asset_id,
       it.long_name, 
       it.model,
       it.manufacturer,
       it.olivier_index,
       ( CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.manufacturer, ' : ', it2.asset_id) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
       
FROM   `olivier_it` it
  LEFT JOIN `it_build_system_HANNA_v2` it2
 ON        it2.rack = it.rack 
 AND       it2.Salle_Origin = it.Salle_Origin
 AND		  it2.manufacturer = it.manufacturer

WHERE it.status_2 is null and (it.rack like '%hana%' OR it.long_name like '%hana%')
ORDER  BY it.rack DESC

### RING

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name, 
        it.model,
        it.manufacturer,
        it.olivier_index,
        ( CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.manufacturer, ' : ', it2.asset_id) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
       
       
FROM   `olivier_it` it
LEFT JOIN `it_build_system_RING_v2` it2 
ON        it2.rack = it.rack 
AND       it2.Salle_Origin = it.Salle_Origin
AND		  it2.manufacturer = it.manufacturer
    
WHERE (it.short_name like '%NA9TCR%' 
OR it.long_name like '%NA9TCR%')
ORDER BY it.rack DESC, it.Salle_Origin

### FEX 

#### Cette query a été lancé apres CMDB-OK-OLD et MISMATCH_TIRET

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name, 
        it.model,
        it.manufacturer,
        it.olivier_index,
        ( CASE WHEN it.status_2 is null and it2.status_2 is not null and it2.asset_id !='0' then Concat(it.manufacturer, ' : ', it2.asset_id) 
        	  WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it2.asset_id ='0') then  Concat(it.manufacturer, ' : No Match') 
        END ) agg
        
       
FROM   `olivier_it` it
LEFT JOIN `IT_Equipment_FEX_v2` it2 on it2.root_fex = (LEFT(it.long_name,8)) and it.long_name != it2.root_fex
    
where it.Fabricant like '%cisco%' and it2.root_fex is not null
ORDER BY it.rack DESC, it.Salle_Origin

## `'CMDB-OK-OLD'`

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name, 
        it.model,
        it.manufacturer,
       it.olivier_index
       
       
FROM   `olivier_it` it
LEFT JOIN `assets_filtree_v2` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.long_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.long_name 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name
where it.status_2 is NULL and (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

## `'MISMATCH_TIRET'`

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       it.long_name,
       aa.nom_long,
       it.short_name,
       aa.nom_court,
        it.model,
        it.manufacturer,
        it.olivier_index
        
FROM   `olivier_it` it
INNER JOIN `assets_filtree_v2` aa
ON UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.long_name,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_long,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.long_name,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.short_name,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_long,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.short_name,'_',''),'-','')) 

ORDER BY it.rack DESC, it.Salle_Origin



## `HITASHI`

In [ ]:
SELECT it.rack,
       it.Salle_Origin,
       it.status_2,
       it.asset_id,
       aa.asset_id,
       it.long_name,
       aa.nom_long,
       it.short_name,
       aa.nom_court,
        it.model,
        it.manufacturer,
        it.olivier_index,
        (
				CASE WHEN aa.asset_id is null and it.status_2 is null then 'Hitashi : No match'
    			WHEN aa.asset_id is not null and it.status_2 is null then concat('HDS : ',aa.asset_id)
    			END 
    			) agg
        
FROM   `olivier_it` it
INNER JOIN `IT_Equipment_cleaning_Hitachi_v2` hit on hit.it_nom = it.long_name
LEFT JOIN `assets_filtree_v2` aa on UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-','')) = hit.clean_name

WHERE it.Fabricant like '%Hitachi%'
ORDER BY it.rack DESC, it.Salle_Origin